<em>Ansys chemkin</em> offers some idealized reactor models commonly used for studying chemical processes and for developing reaction mechanisms. The <i><u>batch reactor model</u></i> is a transient 0D numerical portrayal of the <u>closed</u> <u>homogeneous/perfectly-mixed</u> gas-phase reactor. There are two basic types of batch reactor models: <ol><em>constrained-pressure</em></ol> <ol><em>constrained-volume</em></ol>You can choose either to <u>specify the reactor temperature (as a fixed value or by a piecewise-linear profile)</u> or to <u>solve the energy conservation equation</u> for each reactor type. In total, you get <u>four variations</u> out of the base batch reactor model. 

You can find detailed description of <em>chemkin</em> batch (closed homogeneous) reaction model in <em>Ansys Chemkin</m> <u>Theory manual</u> on the <a href='https://ansyshelp.ansys.com/account/secured?returnurl=/Views/Secured/prod_page.html?pn=Chemkin&pid=ChemkinPro&lang=en'><i>Ansys Help</i> website</a>. For reactor input parameters that do not have a Python API, you can use the <code>setkeyword</code> method to assign the parameter value with the appropriate <u>keyword phrase</u>. <em>Ansys Chemkin</m> <u>Input manual</u> lists all the keywords for all <em>chemkin</em> reactor models. Note that <strong>PyChemkin</strong> does not accept every keywords available to a reactor model.    

<i><u>Rapid Compression Machine (RCM)</u></i> is often employed to study fuel auto-ignition at high temperature and high pressure conditions that are comparible to the engine operating environments. The fuel-air mixture inside the RCM chamber is at relatively low pressure and temperature initially. The gas mixture is then suddenly compressed causing both the pressure and the temperature of the mixture to increase rapidly. The reactor/chamber pressure is monitored to identify the onset of auto-ignition after the compression stopped. This tutorial models the RCM as a <i><u>constrained-volume batch reactor</u></i>, and the compression process is simulated by a predetermined time-volume profile.      

You start with importing these required Python packages: <code>os</code>, <code>chemkin</code>, <code>numppy</code>, and <code>matplotlib</code> (for plotting purposes). In addition, you will have to import the <code>chemkin.batchreactor</code> module to access all the batch reactor models in <strong>PyChemkin</strong>.  

In [ ]:
import os
import chemkin as ck                                 # Chemkin
from chemkin import Color
# chemkin batch reactor models (transient)
from chemkin.batchreactors.batchreactor import (
    GivenVolumeBatchReactor_EnergyConservation,
)
%matplotlib inline
import matplotlib.pyplot as plt                                 # plotting
import numpy as np                                              # number crunching

# check working directory
current_dir = os.getcwd()
print("current working directory: " + current_dir)
# set verbose mode
ck.setverbose(True)

<h5>Set up the <em>Chemistry Set</em></h5>Create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the GRI 3.0 mechanism for methane combustion is used to create the <code>MyGasMech</code> <em>Chemistry</em> object. Include and preprocess the transport data so you can calculate and plot the transport property profile from the solution. 

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = os.path.join(ck.ansys_dir, "reaction", "data")
mechanism_dir = data_dir
# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(label="GRI 3.0")
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = os.path.join(mechanism_dir, "grimech30_chem.inp")
MyGasMech.thermfile = os.path.join(mechanism_dir, "grimech30_thermo.dat")
MyGasMech.tranfile = os.path.join(mechanism_dir, "grimech30_transport.dat")
# preprocess the mechanism files
iError = MyGasMech.preprocess()

<h5>Define the fuel-air mixture</h5>The fuel used in this tutorial is pure methane. Use the <i><u>equivalence ratio</u></i> method to create a fuel-lean mixture <code>premixed</code> with an equivalence ratio of 0.7.

In [ ]:
# create a premixed fuel-oxidizer mixture by assigning the equivalence ratio
# create the fuel mixture
fuelmixture = ck.Mixture(MyGasMech)
# set fuel composition
fuelmixture.X = [("CH4", 1.0)]
# setting pressure and temperature is not required in this case
fuelmixture.pressure = 5.0 * ck.Patm
fuelmixture.temperature = 1500.0
# create the oxidizer mixture: air
air = ck.Mixture(MyGasMech)
air.X = [("O2", 0.21), ("N2", 0.79)]
# setting pressure and temperature is not required in this case
air.pressure = 5.0 * ck.Patm
air.temperature = 1500.0
# create the premixed mixture to be defined
premixed = ck.Mixture(MyGasMech)
# products from the complete combustion of the fuel mixture and air
products = ["CO2", "H2O", "N2"]
# species mole fractions of added/inert mixture. can also create an additives mixture here
add_frac = np.zeros(MyGasMech.KK, dtype=np.double)   # no additives: all zeros
iError = premixed.XbyEquivalenceRatio(MyGasMech, fuelmixture.X, air.X, add_frac, products, equivalenceratio=0.7)
if iError != 0:
    raise RuntimeError

List the composition of <code>premixed</code> so you can verify the initial gas composition in the RCM later. 

In [ ]:
# list the composition of the premixed mixture
premixed.listcomposition(mode="mole")

Set the temperature (800K) and the pressure (3 atm) of <code>premixed</code>. These values will also be used to set the initial conditions of the batch reactor aka the RCM.  

In [ ]:
# set mixture temperature and pressure (equivalent to setting the initial temperature and pressure of the reactor)
premixed.temperature = 800.0
premixed.pressure = 3.0 * ck.Patm

<h5>Create the Batch Reactor Object</h5>Instantiate a <code>GivenVolumeBatchReactor_EnergyConservation</code> batch reactor object <code>MyCONV</code> to represent the RCM. <strong>PyChemkin</strong> requires every <em>Batch Reactor</em> object to be associated with a <em>Mixture</em>. The <em>Mixture</em> implicitly links the <em>Chemistry Set</em> (gas-phase mechanism and properties) to the batch reactor object. Additionally, it also defines the initial conditions (pressure, temperature, volume, and gas composition) of the reactor.  

In [ ]:
# Rapid Compression Machine
# create a constant volume batch reactor (with energy equation)
#
MyCONV = GivenVolumeBatchReactor_EnergyConservation(premixed, label="RCM")

<h5>Set/Verify Reactor Initial Conditions</h5>List the initial gas composition inside the batch reactor <code>MyCONV</code>. You can compare this list to the <code>premixed</code> composition earlier to verify that they are exactly the same.  

In [ ]:
# show initial gas composition inside the reactor
MyCONV.listcomposition(mode="mole")

Since no volume is given to <code>premixed</code>, you have to specify the reactor volume here. 

In [ ]:
# set other reactor properties
# reactor volume [cm3]
MyCONV.volume = 10.0  

Provide a simulation end time [sec] that is long enough to allow the gas mixture to auto-ignition for all cases. 

In [ ]:
# simulation end time [sec]
MyCONV.time = 0.1 

<h5>Set Up the Chamber Volume Profile</h5>Use the <code>setvolumeprofile</code> method and a series of time-volume data pairs to set up a piecewise-linear reactor volume profile. 

In [ ]:
# set RCM volume profile
# number of profile data points
npoints = 3
# position array of the profile data
t = np.zeros(npoints, dtype=np.double)
# value array of the profile data
volprofile = np.zeros_like(t, dtype=np.double)
# set reactor volume data points
t = [0.0, 0.01, 2.0]  # [sec]
volprofile = [10.0, 4.0, 4.0]  # [cm3]
# set the volume profile
MyCONV.setvolumeprofile(t, volprofile)

You can use the <code>phrasehints</code> method  to search for keywords that are related to the phrase. Use the <code>keywordhints</code> method or search the <i><u>Input manual</u></i> to learn about the definition and the syntax of a keyword. 

In [ ]:
# get information about input parameters
ck.phrasehints("heat loss")

Use the <code>setkeyword</code> method to provide supplementary input parameters. Here the heat exchange between the reactor and the surroundings is set to 0.0 by using the keyword \"QLOS\".

In [ ]:
# add input parameters using keywords
# set adiabatic reactor 
MyCONV.setkeyword("QLOS", 0.0e0)

<h5>Set Output Control Parameters</h5>

By default, time intervals for both print and save solution are 100th of the simulation end time. In this case $$dt=time/100=0.001.$$ You can change them to different values.

In [ ]:
# output controls
# set timestep between saving solution
MyCONV.timestepforsavingsolution = 0.01

You can turn on the adaptive solution saving to resolve the steep variations in the solution profile. Here additional solution data point will be saved for every <u>100K</u> change in gas <u>temperature</u>.  

In [ ]:
# turn ON adaptive solution saving
MyCONV.adaptivesolutionsaving(mode=True, value_change=100, target="TEMPERATURE")
# turn OFF adaptive solution saving
# MyCONV.adaptivesolutionsaving(mode=False)

<h5>Set Solver Parameters</h5>Changing solver tolerances sometimes is necessary; especially when you expect large and sudden variation in the solution. 

In [ ]:
# set tolerance
MyCONV.settolerances(absolute_tolerance=1.0e-10, relative_tolerance=1.0e-8)
# get solver parameters
ATOL, RTOL = MyCONV.tolerances
print(f"default absolute tolerance = {ATOL}")
print(f"default relative tolerance = {RTOL}")

You can instruct the solver to try harder to look for non-negative solution values.

In [ ]:
# turn on the force non-negative solutions option in the solver
MyCONV.forcenonnegative = True

<h5>Define Auto-Ignition</h5>The <code>batch reactor</code> looks for the inflection point in the gas temperature profile as the indication of an auto-ignition. You can choose a different auto-ignition definition.  

Use the <code>showignitiondefinition</code> method to see all available options.

In [ ]:
# specify the ignition definitions
ck.showignitiondefinition()

Normally you should keep the default temperature inflection point definition.

In [ ]:
MyCONV.setignitiondelay(method="T_inflection")

Verify the changes you made to some of the options. 

In [ ]:
# show solver option
print(f"timestep between solution printing: {MyCONV.timestepforprintingsolution}")
# show timestep between printing solution
print(f"forced non-negative solution values: {MyCONV.forcenonnegative}")

You can use the <code>showkeywordinputlines</code> method to verify all the supplementary keywords are included correctly before running the simulation. 

In [ ]:
# show the additional keywords given by user
MyCONV.showkeywordinputlines()

<h5>Run the Simulations</h5>Use the <code>run</code> method to run the reactor model and check the run status. If the run failed, you can check the text output file '<i>all0D_*.out</i>' in the working directory for additional imformation about the causes. You can find the working directory using the <code>os.getcwd()</code> command.

In [ ]:
# run the CONV reactor model 
runstatus = MyCONV.run()
# check run status
if runstatus != 0:
    # run failed!
    print(Color.RED + ">>> RUN FAILED <<<", end=Color.END)
    exit()
# run success!
print(Color.GREEN + ">>> RUN COMPLETED <<<", end=Color.END)

<h5>Post-Process the Solution</h5>Once the simulation is completed successfully, you can begin to post-process the results. The first step is to extract and parse the solutions residing in the memory by using the <code>processsolution</code> method.

In [ ]:
# post-process the solutions
MyCONV.processsolution()

You can then use the <code>getnumbersolutionpoints</code> to obtain the size of the solution profile. This piece of information allows you to properly dimension the arrays to hold the solution profiles.

In [ ]:
# get the number of solution time points 
solutionpoints = MyCONV.getnumbersolutionpoints()
print(f"number of solution points = {solutionpoints}")

There are two types of solution data from the <i>batch reactor models<i> in <strong>PyChemkin</strong>: <ol>The <i><u>raw solution profiles</u></i> straight from the <i>batch reactor mode</i> output.</ol><ol>The <i><u>solution mixture profile</u></i> that is a sequence of <em>Mixture</em> objects derived from the raw solutions corresponding to each time point.</ol> 


The <i><u>raw solution profiles</u></i> are time sequencea of these selected solution variables: <u>time</u> [sec], <u>temperature</u> [K], <u>pressure</u> [dynes/cm2], <u>volume</u> [cm3], and <u>species mass fractions</u>. You can use the <code>getsolutionvariableprofile</code> to get them.

Get the <u>time values</u> of the solution time points. 

In [ ]:
# get the time profile
timeprofile = MyCONV.getsolutionvariableprofile("time")

Get the reactor <u>temperature</u> profile. 

In [ ]:
# get the temperature profile 
tempprofile = MyCONV.getsolutionvariableprofile("temperature")

and the <u>volume</u> profile. 

In [ ]:
# get the volu e profile 
volprofile = MyCONV.getsolutionvariableprofile("volume")

You can also extract the <u>mass fraction</u> of a specific gas species.

In [ ]:
# get CH4 mass fraction profile
# CH4massfraction = MyCONV.getsolutionvariableprofile("CH4")

For additional solution variables and properties, you can derive them from the <i><u>solution mixture profile</u></i>.

Prepare the arrays of variables of interest. Use the <code>solutionpoints</code> to size the arrays if needed. 

In [ ]:
#
# more involving post-processing by using Mixtures
#
# mass for validation purpose
massprofile = np.zeros_like(timeprofile, dtype=np.double) 
# create arrays for CH4 mole fraction, CH4 ROP, and mixture viscosity 
CH4profile = np.zeros_like(timeprofile, dtype=np.double)
CH4ROPprofile = np.zeros_like(timeprofile, dtype=np.double) 
viscprofile = np.zeros_like(timeprofile, dtype=np.double) 
CurrentROP = np.zeros(MyGasMech.KK, dtype=np.double)
# find CH4 species index 
CH4_index = MyGasMech.getspecindex("CH4")

Loop over all time points to fetch the <i><u>solution mixture</u></i> using the <code>getsolutionmixtureatindex(time_sequence)</code> and derive the values of variable at each time point. You can use the <code>getsolutionmixture(time_value)</code> to get the solution mixture at the time specified (interpolated from solution values at neighboring actual time points).

In [ ]:
# loop over all solution time points
for i in range(solutionpoints):
    # get the mixture at the time point
    solutionmixture = MyCONV.getsolutionmixtureatindex(solution_index=i)
    # get gas density [g/cm3]
    den = solutionmixture.RHO
    # reactor mass [g]
    massprofile[i] = den * volprofile[i]
    # get CH4 mole fraction profile
    CH4profile[i] = solutionmixture.X[CH4_index]
    # get CH4 ROP profile 
    currentROP = solutionmixture.ROP()
    CH4ROPprofile[i] = currentROP[CH4_index]
    # get mixture vicosity profile 
    viscprofile[i] = solutionmixture.mixtureviscosity()

<h5>Validate the Solution</h5>Even the simulation is completed without any error, you should validate the results before moving forward, especially when it is the first time you use this reactor model. The <i>batch reactor model</i> <code>MyCONV</code> is a closed system so you should at least verify that the reactor mass remains constant during the course of the simulation. It is inevitabe that you will see small fluctuations in the total mass profile. However, as long as the maximum noise level is small enough, the results should be find. If you find the mass variations are too large to be acceptable, you can use smaller tolerance values and/or adjust some solver parameters such as the <i>maximum solver time step size</i> and re-run the simulation.   

In [ ]:
# validation 
del_mass = np.zeros_like(timeprofile, dtype=np.double)
mass0 = massprofile[0]
for i in range(solutionpoints):
    del_mass[i] = abs(massprofile[i] - mass0)
# 
print(f">>> maximum magnitude of reactor mass deviation = {np.max(del_mass)} [g]")

<h5>Find Ignition Delay Time</h5>You can use the <code>getignitiondelay</code> method to obtain the ignition delay time if it is available. Remember to subtract the compression duration from the ignition delay time. 

In [ ]:
# get ignition delay time (need to deduct the initial compression time = 0.01 [sec])
delaytime = MyCONV.getignitiondelay() - 0.01 * 1.0e3
print(f"ignition delay time = {delaytime} [msec]")

<h5>Plot the Solutions</h5>Plot the time profiles of selected <i>raw</i> and <i>mixture-derived</i> variables.

In [ ]:
# plot the profiles
plt.subplots(2, 2, sharex="col", figsize=(12, 6))
plt.subplot(221)
plt.plot(timeprofile, tempprofile, "r-")
plt.ylabel("Temperature [K]")
plt.subplot(222)
plt.plot(timeprofile, CH4profile, "b-")
plt.ylabel("CH4 Mole Fraction")
plt.subplot(223)
plt.plot(timeprofile, CH4ROPprofile, "g-")
plt.xlabel("time [sec]")
plt.ylabel("CH4 Production Rate [mol/cm3-sec]")
plt.subplot(224)
plt.plot(timeprofile, viscprofile, "m-")
plt.xlabel("time [sec]")
plt.ylabel("Mixture Viscosity [g/cm-sec]")
plt.show()